In [ ]:
import numpy as np
import pandas as pd
import keras
import os
from matplotlib import pyplot as plt
% matplotlib inline
from keras.datasets import mnist

#build the gan
#first import all the shit from Keras

from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras import initializers

#we will feed in a vector with 100 (or however many we want) elements of random noise
noise_dim = 100

In [ ]:
#generator
#del gen
gen = Sequential()
gen.add(Dense(128, activation='relu',
              input_shape=(noise_dim,),
              kernel_initializer='he_normal'))
gen.add(Dropout(0.05))  #dropout helps a little, but also trains a little slower
gen.add(Dense(784, activation='tanh',
              kernel_initializer='he_normal'))
gen.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
#discriminator
#del dis
dis = Sequential()
dis.add(Dense(128, activation='relu',
              input_shape=(784,),
              kernel_initializer='he_normal'))
#dis.add(Dense(128, activation= 'selu',kernel_initializer= 'he_normal'))
dis.add(Dense(1, activation='sigmoid',
              kernel_initializer='he_normal'))
dis.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
#combined network
#del gan
dis.trainable = False
ganInput = Input(shape=(noise_dim,))
x = gen(ganInput)
ganOutput = dis(x)
gan = Model(inputs = ganInput, outputs = ganOutput)
gan.compile(loss='binary_crossentropy',optimizer='adam')


In [ ]:
#plot the generated images

def plotGeneratedImages(examples=100, dim=(10, 10), figsize=(9, 9)):
    noise = np.random.normal(0, 1, size=[examples, noise_dim])
    generatedImages = gen.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()


In [ ]:
def plotLossesplotLoss():
    plt.figure(figsize=(15, 8))
    plt.plot(DisLossHist, label='Discriminitive loss')
    plt.plot(GenLossHist, label='Generative loss')


In [ ]:
#initialize a couple of lists to store the losses of the generator and discriminator (useful for plotting)
DisLossHist = []
GenLossHist = []

currentPath = os.path.abspath('.')
(x_train, y_train), (x_test, y_test) = mnist.load_data(
    currentPath + "/data/mnist.npz")


plt.imshow(x_train[0], cmap='gray')
plt.title('Class ' + str(y_train[0]))
num_classes = 10
input_unit_size = 28 * 28


##归一化数据
x_train = x_train.reshape(x_train.shape[0], input_unit_size)
x_test = x_test.reshape(x_test.shape[0], input_unit_size)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

def train(epochs=1, batchSize=128):
    batchCount = int(x_train.shape[0] / batchSize)

    np.random.shuffle(x_train)

    for e in range(epochs):
        if e % 2 == 0:
            print("epoch ", e)
        for b in range(batchCount):
            #if b % 20 == 0:
            #    print("step ",b)

            #first pass through the loop, b is 0!
            noise = np.random.normal(0, 1, size=[batchSize, noise_dim])
            imageBatch = x_train[b * batchSize:(b + 1) * batchSize, :]

            #generate fake images
            generatedImages = gen.predict(noise)

            #append a batch of real images to fake images
            discriminatorInputs = np.vstack((imageBatch, generatedImages))

            #create labels for the disciminator inputs
            yDis = np.zeros(2 * batchSize)
            yDis[:batchSize] = 0.9

            #train Discriminator
            dis.trainable = True
            dLoss = dis.train_on_batch(discriminatorInputs, yDis)

            #train Generator
            dis.trainable = False
            yGen = np.ones(batchSize)
            gLoss = gan.train_on_batch(noise, yGen)

        DisLossHist.append(dLoss)
        GenLossHist.append(gLoss)
    print("Done")


In [ ]:
%%time
train(epochs = 100, batchSize = 128)

In [ ]:
plotLossesplotLoss()

In [ ]:
plotGeneratedImages()

In [ ]:
def saveModels():
    gen.save('gan_generator_twos.h5')
    dis.save('dis_generator_twos.h5')

saveModels()
